In [2]:
# to register
from pt.modeling.meta_arch.rcnn import GuassianGeneralizedRCNN
from pt.modeling.proposal_generator.rpn import GuassianRPN
from pt.modeling.roi_heads.roi_heads import GuassianROIHead
import pt.data.datasets.builtin
from pt.modeling.backbone.vgg import build_vgg_backbone
from pt.modeling.anchor_generator import DifferentiableAnchorGenerator


from pt.modeling.meta_arch.ts_ensemble import EnsembleTSModel
from detectron2.checkpoint import DetectionCheckpointer
import torch


/home/superorange5/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:803: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


## load config

In [8]:
from detectron2.config import get_cfg
from pt import add_config
from FLpkg import add_config as fl_add_config

def setup(config_file):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_config(cfg)
    fl_add_config(cfg)
    cfg.merge_from_file(config_file)
    #cfg.merge_from_list(args.opts)
    
    #default_setup(cfg, args)
#     cfg.SOLVER.IMG_PER_BATCH_LABEL = 64
#     cfg.SOLVER.IMG_PER_BATCH_UNLABEL = 64

    cfg.freeze()
    return cfg

## load model

In [9]:
from detectron2.modeling import build_model
from pt.engine.trainer import PTrainer
from detectron2.checkpoint import DetectionCheckpointer




def load_TSmodel(cfg_path, model_path):
    cfg = setup(cfg_path)
    #cfg.defrost()
    #cfg.MODEL.WEIGHTS = model_path
    
    Trainer =PTrainer
    model = Trainer.build_model(cfg)
    model_teacher = Trainer.build_model(cfg)
    ensem_ts_model = EnsembleTSModel(model_teacher, model)    
    
    
    DetectionCheckpointer(ensem_ts_model).resume_or_load(model_path, resume=False)
    
    return ensem_ts_model

In [5]:
import os
model_folder = 'keep_experiments'

def get_model(dataset_name,model_num):
    if model_num =='final':
        model_name ='model_final.pth'
    else:
        model_name = 'model_{0:07d}.pth'.format(model_num)
    model_path = os.path.join(model_folder,dataset_name,model_name)
    cfg_path = os.path.join(model_folder,dataset_name,'cfg.yaml')
    print(cfg_path)
    print(model_path)
    return load_TSmodel(cfg_path, model_path)

In [27]:
cfg_path = 'configs/FedAvg/FedAvg_ck2b.yaml'
model_path = 'output/FedAvg_ck2b/VOC2007_citytrain1_0/model_final.pth'
model_c2b= load_TSmodel(cfg_path, model_path)

-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [28]:
model_path = 'output/FedAvg_ck2b/VOC2007_kitti1_0/model_final.pth'
model_k2b= load_TSmodel(cfg_path, model_path)

-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [29]:
model_list_ck2b=[model_c2b,model_k2b]


In [81]:

#---------skf2c-------
model_s2c = get_model('s2c', 15999)
model_k2c = get_model('k2c', 19999)
model_f2c = get_model('f2c_1class', 11999) 
model_list_skf2c=[model_s2c,model_k2c,model_f2c]


keep_experiments/s2c/cfg.yaml
keep_experiments/s2c/model_0015999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2c/cfg.yaml
keep_experiments/k2c/model_0019999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/f2c_1class/cfg.yaml
keep_experiments/f2c_1class/model_0011999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [108]:
#--------ck2bdd-------
model_c2b = get_model('c2b', 11999)
model_k2b = get_model('k2b', 'final')
model_list_ck2b=[model_c2b,model_k2b]



keep_experiments/c2b/cfg.yaml
keep_experiments/c2b/model_0011999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2b/cfg.yaml
keep_experiments/k2b/model_final.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [110]:

device = torch.device('cuda:0')
model_c2b
model_c2b = model_c2b.to(device)

In [114]:
next(model_c2b.parameters()).device

device(type='cuda', index=0)

# Avg

In [12]:
def avgWeight(model_list,ratio_list):
    parties = len(model_list)
    model_tmp=[None] * parties
    #optims_tmp=[None] * parties

    for idx, my_model in enumerate(model_list):
        
        model_tmp[idx] = my_model.state_dict()


    for key in model_tmp[0]:    
        #print(key)
        model_avg = 0

        for idx, model_tmp_content in enumerate(model_tmp):     # add each model              
            model_avg += ratio_list[idx] * model_tmp_content[key]
            
        for i in range(len(model_tmp)):  #copy to each model            
            model_tmp[i][key] = model_avg
    for i in range(len(model_list)):    
        model_list[i].load_state_dict(model_tmp[i])
        
    return model_list  #, optims_tmp
    

In [13]:
def avg_model_weight(model_list):
    balance = 1/len(model_list)
    ratio_list= [balance] * len(model_list)
    
    avg_model=avgWeight(model_list,ratio_list)
    return avg_model


## avg skf2c

In [84]:
mGPUs=False
avg_model = avg_model_weight(model_list_skf2c)

In [86]:
output_dir = 'models/avg_skf2c'
save_name = os.path.join(output_dir, 'pt_skf2c_AVG.pth')
torch.save(avg_model[0].state_dict(), save_name)

## avg ck2b

In [30]:
mGPUs=False
avg_model = avg_model_weight(model_list_ck2b)



In [31]:
output_dir = 'models/avg_ck2b'
save_name = os.path.join(output_dir, 'pt_ck2b_AVG_round3-2.pth')
torch.save(avg_model[0].state_dict(), save_name)

In [4]:
load_avgmodel = torch.load("keep_experiments/FedPT_skf2c/pt_skf2c_AVG.pth")
load_avgmodel['modelStudent.proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 1.0949e-02,  3.6106e-02, -8.3926e-01, -5.0064e-01, -1.8014e+00,
        -2.1438e+00, -1.3580e+00, -1.4464e+00,  5.1012e-03,  1.5298e-02,
        -2.6144e-01, -4.8957e-01, -9.3746e-01, -1.1258e+00, -8.0347e-01,
        -8.5810e-01, -8.9569e-04,  4.2415e-02, -3.1489e-01, -1.2556e+00,
        -1.1018e+00, -1.6336e+00, -8.8184e-01, -1.0403e+00, -5.7032e-03,
        -9.3344e-03, -1.0566e-01, -3.0487e-02, -3.5011e-01, -3.8200e-01,
        -2.8333e-01, -2.9011e-01, -7.7315e-03, -3.0739e-03, -4.0304e-02,
        -9.7155e-02, -4.3825e-01, -4.5539e-01, -3.3827e-01, -3.3164e-01,
        -1.2578e-02, -8.0955e-04, -3.0547e-02, -7.0892e-02, -2.8793e-01,
        -3.0009e-01, -2.4454e-01, -2.2057e-01,  5.6443e-05, -5.4561e-03,
        -3.9191e-02, -2.5039e-02, -9.7121e-02, -1.0738e-01, -8.8014e-02,
        -8.9471e-02, -5.1024e-03, -8.9636e-03, -7.9096e-02, -1.0914e-01,
        -1.9555e-01, -1.8664e-01, -1.6924e-01, -1.5845e-01,  1.1864e-02,
         7.9556e-04, -3.1962e-02, -3.8323e-02, -1.1

### check weight

In [41]:
model_c2b.state_dict()['modelStudent.proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 4.9567e-03,  4.1854e-02, -8.4723e-01, -5.3333e-01, -1.7204e+00,
        -1.9618e+00, -1.1261e+00, -1.2538e+00,  7.7744e-03,  2.2535e-02,
        -2.2750e-01, -4.8190e-01, -7.0024e-01, -8.2286e-01, -5.7922e-01,
        -6.0722e-01, -3.5713e-02,  6.8824e-02, -2.9323e-01, -1.2816e+00,
        -9.5808e-01, -1.4632e+00, -6.4602e-01, -8.0602e-01, -1.1137e-03,
         4.0937e-03, -1.2114e-01, -3.2870e-02, -2.6557e-01, -2.5672e-01,
        -2.0734e-01, -1.9663e-01,  4.5403e-03, -7.9842e-03, -4.0569e-03,
        -1.0426e-01, -3.1593e-01, -3.0426e-01, -2.6025e-01, -2.3013e-01,
        -7.1750e-03,  1.7197e-03, -2.0883e-02, -4.9887e-02, -1.4983e-01,
        -1.3809e-01, -1.3263e-01, -1.1133e-01,  3.7207e-03,  9.6099e-04,
        -4.0151e-02, -2.8066e-02, -9.2451e-02, -9.5428e-02, -8.6735e-02,
        -7.9944e-02,  5.4622e-03,  4.5548e-03, -4.3455e-02, -6.1940e-02,
        -1.1308e-01, -1.0045e-01, -1.0183e-01, -8.1233e-02,  2.0255e-03,
         4.3722e-03, -2.2436e-02, -2.0772e-02, -5.6

In [13]:
model_k2b.state_dict()['modelStudent.proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 1.4360e-02,  3.2181e-02, -7.3846e-01, -3.9803e-01, -2.4172e+00,
        -2.5198e+00, -1.8527e+00, -1.8478e+00,  9.1398e-03,  3.1058e-02,
        -2.1960e-01, -4.9665e-01, -1.1078e+00, -1.2044e+00, -9.1253e-01,
        -8.9858e-01,  9.4522e-03,  6.7336e-02, -1.7252e-01, -1.1749e+00,
        -1.5474e+00, -1.9690e+00, -1.1923e+00, -1.2674e+00, -7.1267e-03,
        -1.8707e-03, -1.2198e-01, -1.9326e-02, -5.1146e-01, -4.7497e-01,
        -4.2590e-01, -3.6520e-01,  7.1992e-03,  1.5896e-04, -5.8629e-02,
        -1.3048e-01, -5.8289e-01, -5.4069e-01, -4.5232e-01, -3.6021e-01,
         6.4223e-04, -2.7411e-04, -1.5138e-02, -7.5643e-02, -2.8181e-01,
        -2.4595e-01, -2.2867e-01, -1.5775e-01, -2.8776e-03, -1.4509e-03,
        -9.4464e-02, -5.7442e-02, -1.9346e-01, -1.7289e-01, -1.6273e-01,
        -1.2784e-01,  6.1003e-02,  1.2919e-02, -1.5004e-01, -1.6835e-01,
        -2.4450e-01, -2.1725e-01, -2.1886e-01, -1.6044e-01, -2.7775e-03,
        -3.4089e-04, -1.0674e-02, -3.2906e-02, -1.0

In [19]:
avg_model[0].state_dict()['modelStudent.proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 9.6583e-03,  3.7017e-02, -7.9285e-01, -4.6568e-01, -2.0688e+00,
        -2.2408e+00, -1.4894e+00, -1.5508e+00,  8.4571e-03,  2.6796e-02,
        -2.2355e-01, -4.8927e-01, -9.0401e-01, -1.0136e+00, -7.4587e-01,
        -7.5290e-01, -1.3130e-02,  6.8080e-02, -2.3288e-01, -1.2282e+00,
        -1.2528e+00, -1.7161e+00, -9.1913e-01, -1.0367e+00, -4.1202e-03,
         1.1115e-03, -1.2156e-01, -2.6098e-02, -3.8852e-01, -3.6585e-01,
        -3.1662e-01, -2.8091e-01,  5.8698e-03, -3.9126e-03, -3.1343e-02,
        -1.1737e-01, -4.4941e-01, -4.2248e-01, -3.5628e-01, -2.9517e-01,
        -3.2664e-03,  7.2280e-04, -1.8010e-02, -6.2765e-02, -2.1582e-01,
        -1.9202e-01, -1.8065e-01, -1.3454e-01,  4.2153e-04, -2.4497e-04,
        -6.7308e-02, -4.2754e-02, -1.4296e-01, -1.3416e-01, -1.2473e-01,
        -1.0389e-01,  3.3233e-02,  8.7368e-03, -9.6750e-02, -1.1515e-01,
        -1.7879e-01, -1.5885e-01, -1.6035e-01, -1.2084e-01, -3.7602e-04,
         2.0156e-03, -1.6555e-02, -2.6839e-02, -8.0

In [14]:
k2bmodel = torch.load("keep_experiments/k2b/model_final.pth")
k2bmodel['model']['modelStudent.proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 1.4360e-02,  3.2181e-02, -7.3846e-01, -3.9803e-01, -2.4172e+00,
        -2.5198e+00, -1.8527e+00, -1.8478e+00,  9.1398e-03,  3.1058e-02,
        -2.1960e-01, -4.9665e-01, -1.1078e+00, -1.2044e+00, -9.1253e-01,
        -8.9858e-01,  9.4522e-03,  6.7336e-02, -1.7252e-01, -1.1749e+00,
        -1.5474e+00, -1.9690e+00, -1.1923e+00, -1.2674e+00, -7.1267e-03,
        -1.8707e-03, -1.2198e-01, -1.9326e-02, -5.1146e-01, -4.7497e-01,
        -4.2590e-01, -3.6520e-01,  7.1992e-03,  1.5896e-04, -5.8629e-02,
        -1.3048e-01, -5.8289e-01, -5.4069e-01, -4.5232e-01, -3.6021e-01,
         6.4223e-04, -2.7411e-04, -1.5138e-02, -7.5643e-02, -2.8181e-01,
        -2.4595e-01, -2.2867e-01, -1.5775e-01, -2.8776e-03, -1.4509e-03,
        -9.4464e-02, -5.7442e-02, -1.9346e-01, -1.7289e-01, -1.6273e-01,
        -1.2784e-01,  6.1003e-02,  1.2919e-02, -1.5004e-01, -1.6835e-01,
        -2.4450e-01, -2.1725e-01, -2.1886e-01, -1.6044e-01, -2.7775e-03,
        -3.4089e-04, -1.0674e-02, -3.2906e-02, -1.0

In [39]:
c2bmodel = torch.load("keep_experiments/c2b/model_0011999.pth")
c2bmodel['model']['modelStudent.proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 4.9567e-03,  4.1854e-02, -8.4723e-01, -5.3333e-01, -1.7204e+00,
        -1.9618e+00, -1.1261e+00, -1.2538e+00,  7.7744e-03,  2.2535e-02,
        -2.2750e-01, -4.8190e-01, -7.0024e-01, -8.2286e-01, -5.7922e-01,
        -6.0722e-01, -3.5713e-02,  6.8824e-02, -2.9323e-01, -1.2816e+00,
        -9.5808e-01, -1.4632e+00, -6.4602e-01, -8.0602e-01, -1.1137e-03,
         4.0937e-03, -1.2114e-01, -3.2870e-02, -2.6557e-01, -2.5672e-01,
        -2.0734e-01, -1.9663e-01,  4.5403e-03, -7.9842e-03, -4.0569e-03,
        -1.0426e-01, -3.1593e-01, -3.0426e-01, -2.6025e-01, -2.3013e-01,
        -7.1750e-03,  1.7197e-03, -2.0883e-02, -4.9887e-02, -1.4983e-01,
        -1.3809e-01, -1.3263e-01, -1.1133e-01,  3.7207e-03,  9.6099e-04,
        -4.0151e-02, -2.8066e-02, -9.2451e-02, -9.5428e-02, -8.6735e-02,
        -7.9944e-02,  5.4622e-03,  4.5548e-03, -4.3455e-02, -6.1940e-02,
        -1.1308e-01, -1.0045e-01, -1.0183e-01, -8.1233e-02,  2.0255e-03,
         4.3722e-03, -2.2436e-02, -2.0772e-02, -5.6

### eval avg weight by PTrainer

* use train_net.py --eval-only MODEL.WEIGHTS xxx

### try load by detection checkpointer

In [93]:
cfg = setup("keep_experiments/c2b/cfg.yaml")
cfg.defrost()


In [ ]:


Trainer =PTrainer
model = Trainer.build_model(cfg)
model_teacher = Trainer.build_model(cfg)
ensem_ts_model = EnsembleTSModel(model_teacher, model)
DetectionCheckpointer(ensem_ts_model, save_dir="outputs/avgbddtest").resume_or_load("keep_experiments/c2b/model_0011999.pth", resume=False)

In [95]:
ensem_ts_model.modelTeacher.state_dict()['proposal_generator.rpn_head.anchor_deltas.bias']

tensor([-6.5627e-03,  3.0417e-02, -8.4835e-01, -5.3043e-01, -1.3700e+00,
        -1.6144e+00, -8.4943e-01, -9.8027e-01,  1.7856e-03,  2.0595e-02,
        -2.0001e-01, -4.3966e-01, -5.2200e-01, -6.2199e-01, -4.3529e-01,
        -4.5533e-01, -2.6739e-02,  5.9310e-02, -2.9256e-01, -1.2632e+00,
        -7.1997e-01, -1.1533e+00, -4.8411e-01, -6.2959e-01, -4.2020e-03,
         2.0499e-03, -9.9693e-02, -2.6648e-02, -2.0214e-01, -1.9674e-01,
        -1.6394e-01, -1.5516e-01,  3.4224e-03, -3.5436e-03, -1.7894e-02,
        -9.9878e-02, -2.4250e-01, -2.3280e-01, -2.0532e-01, -1.8185e-01,
        -6.4977e-03,  5.2584e-03, -1.6049e-02, -4.5818e-02, -1.1476e-01,
        -1.0712e-01, -1.0239e-01, -8.6640e-02,  6.6900e-04, -1.1808e-04,
        -2.7552e-02, -2.2804e-02, -6.9459e-02, -6.5970e-02, -6.2301e-02,
        -5.5687e-02, -4.1321e-03,  5.3846e-03, -3.9390e-02, -4.6522e-02,
        -8.4726e-02, -7.3776e-02, -7.7774e-02, -6.1891e-02,  2.1314e-03,
         3.9247e-03, -1.6818e-02, -1.5795e-02, -4.2

In [96]:
ensem_ts_model.modelStudent.state_dict()['proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 4.9567e-03,  4.1854e-02, -8.4723e-01, -5.3333e-01, -1.7204e+00,
        -1.9618e+00, -1.1261e+00, -1.2538e+00,  7.7744e-03,  2.2535e-02,
        -2.2750e-01, -4.8190e-01, -7.0024e-01, -8.2286e-01, -5.7922e-01,
        -6.0722e-01, -3.5713e-02,  6.8824e-02, -2.9323e-01, -1.2816e+00,
        -9.5808e-01, -1.4632e+00, -6.4602e-01, -8.0602e-01, -1.1137e-03,
         4.0937e-03, -1.2114e-01, -3.2870e-02, -2.6557e-01, -2.5672e-01,
        -2.0734e-01, -1.9663e-01,  4.5403e-03, -7.9842e-03, -4.0569e-03,
        -1.0426e-01, -3.1593e-01, -3.0426e-01, -2.6025e-01, -2.3013e-01,
        -7.1750e-03,  1.7197e-03, -2.0883e-02, -4.9887e-02, -1.4983e-01,
        -1.3809e-01, -1.3263e-01, -1.1133e-01,  3.7207e-03,  9.6099e-04,
        -4.0151e-02, -2.8066e-02, -9.2451e-02, -9.5428e-02, -8.6735e-02,
        -7.9944e-02,  5.4622e-03,  4.5548e-03, -4.3455e-02, -6.1940e-02,
        -1.1308e-01, -1.0045e-01, -1.0183e-01, -8.1233e-02,  2.0255e-03,
         4.3722e-03, -2.2436e-02, -2.0772e-02, -5.6

In [47]:
for key in c2bmodel['model']:
    #print(key)
    if(key=='modelStudent.proposal_generator.rpn_head.anchor_deltas.bias'):
        print(c2bmodel['model'][key])
    

tensor([ 4.9567e-03,  4.1854e-02, -8.4723e-01, -5.3333e-01, -1.7204e+00,
        -1.9618e+00, -1.1261e+00, -1.2538e+00,  7.7744e-03,  2.2535e-02,
        -2.2750e-01, -4.8190e-01, -7.0024e-01, -8.2286e-01, -5.7922e-01,
        -6.0722e-01, -3.5713e-02,  6.8824e-02, -2.9323e-01, -1.2816e+00,
        -9.5808e-01, -1.4632e+00, -6.4602e-01, -8.0602e-01, -1.1137e-03,
         4.0937e-03, -1.2114e-01, -3.2870e-02, -2.6557e-01, -2.5672e-01,
        -2.0734e-01, -1.9663e-01,  4.5403e-03, -7.9842e-03, -4.0569e-03,
        -1.0426e-01, -3.1593e-01, -3.0426e-01, -2.6025e-01, -2.3013e-01,
        -7.1750e-03,  1.7197e-03, -2.0883e-02, -4.9887e-02, -1.4983e-01,
        -1.3809e-01, -1.3263e-01, -1.1133e-01,  3.7207e-03,  9.6099e-04,
        -4.0151e-02, -2.8066e-02, -9.2451e-02, -9.5428e-02, -8.6735e-02,
        -7.9944e-02,  5.4622e-03,  4.5548e-03, -4.3455e-02, -6.1940e-02,
        -1.1308e-01, -1.0045e-01, -1.0183e-01, -8.1233e-02,  2.0255e-03,
         4.3722e-03, -2.2436e-02, -2.0772e-02, -5.6

### load avg model

In [96]:
avg_bdd_model = torch.load("models/avg_ck2b/pt_ck2b_AVG.pth")


## evaluation (defaults.test)

In [44]:
model_c2b.modelStudent.state_dict()

tensor([ 4.9567e-03,  4.1854e-02, -8.4723e-01, -5.3333e-01, -1.7204e+00,
        -1.9618e+00, -1.1261e+00, -1.2538e+00,  7.7744e-03,  2.2535e-02,
        -2.2750e-01, -4.8190e-01, -7.0024e-01, -8.2286e-01, -5.7922e-01,
        -6.0722e-01, -3.5713e-02,  6.8824e-02, -2.9323e-01, -1.2816e+00,
        -9.5808e-01, -1.4632e+00, -6.4602e-01, -8.0602e-01, -1.1137e-03,
         4.0937e-03, -1.2114e-01, -3.2870e-02, -2.6557e-01, -2.5672e-01,
        -2.0734e-01, -1.9663e-01,  4.5403e-03, -7.9842e-03, -4.0569e-03,
        -1.0426e-01, -3.1593e-01, -3.0426e-01, -2.6025e-01, -2.3013e-01,
        -7.1750e-03,  1.7197e-03, -2.0883e-02, -4.9887e-02, -1.4983e-01,
        -1.3809e-01, -1.3263e-01, -1.1133e-01,  3.7207e-03,  9.6099e-04,
        -4.0151e-02, -2.8066e-02, -9.2451e-02, -9.5428e-02, -8.6735e-02,
        -7.9944e-02,  5.4622e-03,  4.5548e-03, -4.3455e-02, -6.1940e-02,
        -1.1308e-01, -1.0045e-01, -1.0183e-01, -8.1233e-02,  2.0255e-03,
         4.3722e-03, -2.2436e-02, -2.0772e-02, -5.6

In [45]:
#results = OrderedDict()
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import PascalVOCDetectionEvaluator
from detectron2.evaluation import inference_on_dataset

cfg = setup("keep_experiments/c2b/cfg.yaml")

dataset_name = 'VOC2007_bddvalsmall'
test_data_loader = build_detection_test_loader(cfg, dataset_name)
evaluator = PascalVOCDetectionEvaluator(dataset_name)




In [46]:
#model_c2b.eval()
results_i = inference_on_dataset(model_c2b.modelStudent, test_data_loader, evaluator)


In [47]:
results_i

OrderedDict([('bbox',
              {'AP': 34.43792240729627,
               'AP50': 55.33559113300493,
               'AP75': 42.69925476822028})])

In [54]:
model = model_c2b.modelStudent

In [67]:
data_loader_iter = iter(test_data_loader)
data = data_loader_iter.next()

In [84]:
with torch.no_grad():
    model.eval()
    output = model_c2b.modelStudent(data)


In [90]:
output[0]['instances'].image_size

(720, 1280)

In [107]:
model_c2b

NameError: name 'model_c2b' is not defined

In [101]:
output[0]['instances'].pred_boxes

output[0]['instances'].scores_logists
output[0]['instances'].boxes_sigma
output[0]['instances'].scores

tensor([0.9708, 0.9707, 0.9612, 0.9462, 0.9312, 0.8988, 0.8963, 0.8774, 0.8749,
        0.7561, 0.7531, 0.6275, 0.5939, 0.5390, 0.4565, 0.4178, 0.2566, 0.1998,
        0.1848, 0.1812, 0.1348, 0.1121, 0.0759, 0.0390, 0.0330, 0.0310, 0.0206,
        0.0180, 0.0171, 0.0161, 0.0135, 0.0127, 0.0123, 0.0106, 0.0099, 0.0094],
       device='cuda:0')

In [95]:
output[0]['instances'].get_fields

<bound method Instances.get_fields of Instances(num_instances=36, image_height=720, image_width=1280, fields=[pred_boxes: Boxes(tensor([[9.3210e+02, 3.3852e+02, 1.2210e+03, 4.8170e+02],
        [7.9497e+02, 3.6244e+02, 9.0552e+02, 4.3002e+02],
        [7.5011e+02, 3.6257e+02, 8.2003e+02, 4.1306e+02],
        [3.8800e+01, 3.4999e+02, 1.2780e+02, 4.0018e+02],
        [8.8130e+02, 3.7710e+02, 9.6676e+02, 4.4869e+02],
        [2.4614e+02, 3.4897e+02, 3.4941e+02, 3.9769e+02],
        [5.9876e+02, 3.5402e+02, 6.3575e+02, 3.8532e+02],
        [0.0000e+00, 3.4040e+02, 5.8351e+01, 4.0309e+02],
        [7.3273e+02, 3.6651e+02, 7.7169e+02, 4.0439e+02],
        [6.8310e+02, 3.5948e+02, 7.1826e+02, 3.9106e+02],
        [0.0000e+00, 3.3469e+02, 1.0768e+01, 3.9184e+02],
        [2.0393e+02, 3.4324e+02, 2.5814e+02, 3.8604e+02],
        [2.1476e+02, 3.5879e+02, 3.0001e+02, 4.5576e+02],
        [7.0833e+02, 3.6453e+02, 7.4357e+02, 3.9798e+02],
        [6.9122e+02, 3.6096e+02, 7.3717e+02, 3.9521e+02],
  

In [52]:
for idx, inputs in enumerate(test_data_loader):
    output = model_c2b.modelStudent(input)
    print(output)
    

TypeError: 'method' object is not subscriptable

In [63]:
print(next(g))

4


In [51]:
def get_all_inputs_outputs(data_loader,model):
    for data in data_loader:
        yield data, model(data)


evaluator.reset()
for inputs, outputs in get_all_inputs_outputs(test_data_loader,model_c2b.modelTeacher):
      evaluator.process(inputs, outputs)
eval_results = evaluator.evaluate()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
for idx, inputs in enumerate(test_data_loader):

    outputs = model_teacher(inputs)
    evaluator.process(inputs, outputs)
results = evaluator.evaluate()

# multi-thread test

In [104]:
# from concurrent.futures import ThreadPoolExecutor

def hello(cur_round, first_name, last_name) :
    """Prints a friendly hello with first name and last name"""
    print("-----{}----".format(cur_round))
    print('Hello %s %s!' % (first_name, last_name))
    return "my name is {} {}".format(first_name,last_name)
def main() -> None:    
    
    args_names = []
    
    dataset_list=['kitti','city']
    ROUND=1
    for i,dataset in enumerate(dataset_list):
        args_names.append((ROUND,i,dataset))
    #round_list=({"round":1},{"round":1})
    #args_names = enumerate(dataset_list)
    ## ex1
    with ThreadPoolExecutor() as executor:
        # Using lambda, unpacks the tuple (*f) into hello(*args)
        results = executor.map(lambda f: hello(*f), args_names)
    
    
    ## ex2
#     kwargs_names = (
#         {'first_name': 'Bruce', 'last_name': 'Wayne'},
#         {'first_name': 'Clark', 'last_name': 'Kent'},
#         {'first_name': 'Diana', 'last_name': 'Prince'},
#         {'first_name': 'Barry', 'last_name': 'Allen'},
#     )
#     with ThreadPoolExecutor() as executor:
#         # Using lambda, unpacks the dict (**f) into hello(**kwargs)
#         results = executor.map(lambda f: hello(**f), kwargs_names)
        
    
    return results    
results = main()


-----1---------1----
Hello 0 kitti!

Hello 1 city!


In [98]:
results

<generator object Executor.map.<locals>.result_iterator at 0x7f18e00a7200>

In [105]:
lst = list(results)
len(lst)

2

In [106]:
for r in lst:
    print(r)

my name is 0 kitti
my name is 1 city


In [25]:
import time
import os
from concurrent.futures import ProcessPoolExecutor
from PIL import Image, ImageFilter


args=(('city',1),('kitti',2))
def trainerpool(data,num):
#     img = Image.open(filename)
#     img = img.filter(ImageFilter.GaussianBlur())
#     img.thumbnail(size)
#     img.save(f'{thumb_dir}/{os.path.basename(filename)}')
    time.sleep(1)
    print(f'{data} was processed...')
    
    print(num)
    return num*num


if __name__ == '__main__':
    start = time.perf_counter()

    with ProcessPoolExecutor() as executor:
        results = executor.map(create_thumbnail, args)
   
    for r in results:
        print(r)

('city', 1) was processed...('kitti', 2) was processed...



TypeError: can't multiply sequence by non-int of type 'tuple'

In [6]:
import threading
import time

class TrainerThread(threading.Thread):
    def __init__(self, num):
        threading.Thread.__init__(self)
        self.num = num
        self.model

    def run(self):
        print("Thread", self.num)
        time.sleep(1)
        self.model = self.num*5

# 建立 5 個子執行緒
threads = []
for i in range(5):
    threads.append(TrainerThread(i))
    threads[i].start()

# 主執行緒繼續執行自己的工作
# ...

# 等待所有子執行緒結束
for i in range(5):
    threads[i].join()
    
model_list = []
for i in range(5):
     model_list.append(thread[i].model)
    
mysum=0
for model in model_list:
    mysum+=model
print("average {}".format(mysum/5))

print("Done.")

AttributeError: 'TrainerThread' object has no attribute 'model'